이거는 수질 오염 데이터
- 일단 tidy data..
- binary 처리
- category 화

In [45]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()

    if system_name == "Darwin":
        font_family = "AppleGothic"
    elif system_name == "Windows":
        font_family = "Malgun Gothic"
    else:
        # Linux(colab)
        !apt-get install fonts-nanum - qq > /dev/null
        !fc-cache - fv

        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont

        font_family = "NanumBarunGothic"
    return font_family


# style 설정은 꼭 폰트설정 위에서 합니다.
# style 에 폰트 설정이 들어있으면 한글폰트가 초기화 되어 한글이 깨집니다.
plt.style.use("seaborn")
# 폰트설정
plt.rc("font", family=get_font_family())

# 마이너스폰트 설정
plt.rc("axes", unicode_minus=False)

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'


In [48]:
# 폴더 내 데이터 확인
from glob import glob

file_list = glob('data/water/*.csv')
file_list

['data/water/호소의_수질현황_20220609184509.csv',
 'data/water/호소의_수질현황_20220609184440.csv',
 'data/water/호소의_수질현황_20220609184529.csv',
 'data/water/호소의_수질현황_20220609184413.csv']

In [79]:
# 데이터 가져오기
dfs = [pd.read_csv(name, encoding='cp949') for name in file_list]
dfs[0].head()

,수계별(1),수계별(2),항목,2020.04,2020.05,2020.06,2020.07,2020.08,2020.09,2020.10,2020.11,2020.12
0,한강,화천댐1(파로호),생물화학적산소요구량 BOD (㎎/ℓ),1.1,0.7,1.8,0.9,1.9,1.2,0.6,0.5,1.0
1,한강,화천댐1(파로호),용존산소 DO (㎎/ℓ),12.4,11.1,10.0,9.7,10.2,9.0,8.6,9.8,10.0
2,한강,화천댐1(파로호),수온 (℃),6.6,12.4,15.4,19.5,22.7,18.0,18.2,13.8,10.2
3,한강,화천댐1(파로호),카드뮴 Cd (㎎/ℓ),-,-,0.00020,-,-,0.00000,-,-,0.00000
4,한강,화천댐1(파로호),시안 CN (㎎/ℓ),-,-,0.00,-,-,0.00,-,-,0.00


In [114]:
# 19년도, 20년도 병합
df_19_m = pd.merge(dfs[2], dfs[0])
df_20_m = pd.merge(dfs[1], dfs[3])
df_19_m.columns, df_20_m.columns

(Index(['수계별(1)', '수계별(2)', '항목', '2020.01', '2020.02', '2020.03', '2020.04',
        '2020.05', '2020.06', '2020.07', '2020.08', '2020.09', '2020.10',
        '2020.11', '2020.12'],
       dtype='object'),
 Index(['수계별(1)', '수계별(2)', '항목', '2019.01', '2019.02', '2019.03', '2019.04',
        '2019.05', '2019.06', '2019.07', '2019.08', '2019.09', '2019.10',
        '2019.11', '2019.12'],
       dtype='object'))

In [138]:
# tidy data, 컬럼명 변경
cols = ['강명', '댐명', '항목', '시점', '오염지수']
df_19 = df_19_m.melt(id_vars=['수계별(1)', '수계별(2)', '항목'], var_name='시점')
df_20 = df_20_m.melt(id_vars=['수계별(1)', '수계별(2)', '항목'], var_name='시점')
df_19.columns = cols
df_20.columns = cols
df_20.head()


,강명,댐명,항목,시점,오염지수
0,한강,화천댐1(파로호),생물화학적산소요구량 BOD (㎎/ℓ),2019.01,1.0
1,한강,화천댐1(파로호),용존산소 DO (㎎/ℓ),2019.01,10.3
2,한강,화천댐1(파로호),수온 (℃),2019.01,5.3
3,한강,화천댐1(파로호),카드뮴 Cd (㎎/ℓ),2019.01,-
4,한강,화천댐1(파로호),시안 CN (㎎/ℓ),2019.01,-


In [139]:
# 안쓰는 컬럼 삭제
# del df_19['수계별(1)']
# del df_20['수계별(1)']
# del df_19['시점']
# del df_20['시점']
df_19.columns, df_20.columns


(Index(['강명', '댐명', '항목', '시점', '오염지수'], dtype='object'),
 Index(['강명', '댐명', '항목', '시점', '오염지수'], dtype='object'))

In [140]:
# shape 확인
df_19.shape, df_20.shape

((21168, 5), (20280, 5))

In [141]:
# dtype 확인
df_19.dtypes

강명      object
댐명      object
항목      object
시점      object
오염지수    object
dtype: object

In [144]:
# 수치형으로 타입 변경
df_19['오염지수'] = pd.to_numeric(df_19['오염지수'], errors='coerce', downcast='unsigned')
df_20['오염지수'] = pd.to_numeric(df_20['오염지수'], errors='coerce', downcast='unsigned')
df_19.dtypes, df_20.dtypes

(강명       object
 댐명       object
 항목       object
 시점       object
 오염지수    float64
 dtype: object,
 강명       object
 댐명       object
 항목       object
 시점       object
 오염지수    float64
 dtype: object)

In [145]:
# downcast
ocols = df_19.select_dtypes('object').columns
df_19[ocols] = df_19[ocols].astype('category')
df_19.dtypes

강명      category
댐명      category
항목      category
시점      category
오염지수     float64
dtype: object

In [148]:
# downcast
ocols = df_20.select_dtypes('object').columns
df_20[ocols] = df_20[ocols].astype('category')
df_20.dtypes


강명      category
댐명      category
항목      category
시점      category
오염지수     float64
dtype: object

In [157]:
# info 확인
df_19.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21168 entries, 0 to 21167
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   강명      21168 non-null  category
 1   댐명      21168 non-null  category
 2   항목      21168 non-null  category
 3   시점      21168 non-null  category
 4   오염지수    11787 non-null  float64 
dtypes: category(4), float64(1)
memory usage: 296.8 KB


In [161]:
df_19.head()

,강명,댐명,항목,시점,오염지수
0,한강,화천댐1(파로호),생물화학적산소요구량 BOD (㎎/ℓ),2020.01,0.4
1,한강,화천댐1(파로호),용존산소 DO (㎎/ℓ),2020.01,10.7
2,한강,화천댐1(파로호),수온 (℃),2020.01,6.5
3,한강,화천댐1(파로호),카드뮴 Cd (㎎/ℓ),2020.01,NaN
4,한강,화천댐1(파로호),시안 CN (㎎/ℓ),2020.01,NaN


In [168]:
# 결측치 채워넣기
df_19['오염지수'] = df_19['오염지수'].fillna(0)
df_20['오염지수'] = df_20['오염지수'].fillna(0)
df_19.head()

,강명,댐명,항목,시점,오염지수
0,한강,화천댐1(파로호),생물화학적산소요구량 BOD (㎎/ℓ),2020.01,0.4
1,한강,화천댐1(파로호),용존산소 DO (㎎/ℓ),2020.01,10.7
2,한강,화천댐1(파로호),수온 (℃),2020.01,6.5
3,한강,화천댐1(파로호),카드뮴 Cd (㎎/ℓ),2020.01,0.0
4,한강,화천댐1(파로호),시안 CN (㎎/ℓ),2020.01,0.0


In [183]:
# 그룹핑
df_19_g = df_19.groupby(by=['강명', '항목', '시점'], as_index=False).mean()
df_20_g = df_20.groupby(by=['강명', '항목', '시점'], as_index=False).mean()


In [188]:
df_19_g

,강명,항목,시점,오염지수
0,금강,6가크롬 Cr6+ (㎎/ℓ),2020.01,0.0
1,금강,6가크롬 Cr6+ (㎎/ℓ),2020.02,0.0
2,금강,6가크롬 Cr6+ (㎎/ℓ),2020.03,0.0
3,금강,6가크롬 Cr6+ (㎎/ℓ),2020.04,0.0
4,금강,6가크롬 Cr6+ (㎎/ℓ),2020.05,0.0
...,...,...,...,...
1507,회야강,카드뮴 Cd (㎎/ℓ),2020.08,0.0
1508,회야강,카드뮴 Cd (㎎/ℓ),2020.09,0.0
1509,회야강,카드뮴 Cd (㎎/ℓ),2020.10,0.0
1510,회야강,카드뮴 Cd (㎎/ℓ),2020.11,0.0


In [198]:
# concat 할 수 있는지 확인
((df_20_g['강명'] == df_19_g['강명']) == False).sum()
((df_20_g['항목'] == df_19_g['항목']) == False).sum()

0

In [224]:
# concat
water = pd.concat([df_19_g, df_20_g.loc[:, ['시점', '오염지수']]], axis=1)
water.head()

,강명,항목,시점,오염지수,시점,오염지수
0,금강,6가크롬 Cr6+ (㎎/ℓ),2020.01,0.0,2019.01,0.0
1,금강,6가크롬 Cr6+ (㎎/ℓ),2020.02,0.0,2019.02,0.0
2,금강,6가크롬 Cr6+ (㎎/ℓ),2020.03,0.0,2019.03,0.0
3,금강,6가크롬 Cr6+ (㎎/ℓ),2020.04,0.0,2019.04,0.0
4,금강,6가크롬 Cr6+ (㎎/ℓ),2020.05,0.0,2019.05,0.0


In [225]:
# 데이터 저장
water.to_csv('output/water/water_1920.csv', index=False)